In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_02 import *
import torch.nn.functional as F

## Initial setup

### Data

In [3]:
mpl.rcParams['image.cmap'] = 'gray'

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

In [5]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

### Cross entropy loss

In [6]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [7]:
model = Model(m, nh, 10)

In [8]:
pred = model(x_train)

In [ ]:
def log_softmax(x): return torch.log(x.exp()/(x.exp().sum(-1,keepdim=True)))

In [ ]:
def nll(input, target): return -input[range(target.shape[0]), target].mean()

In [ ]:
loss = nll(log_softmax(pred), y_train)

In [ ]:
def log_softmax(x): return x.sub_(x.exp().sum(-1,keepdim=True).log())

In [ ]:
test_near(nll(log_softmax(pred), y_train), loss)

In [ ]:
def log_softmax(x): return x.sub_(x.logsumexp(-1,keepdim=True))

In [ ]:
test_near(nll(log_softmax(pred), y_train), loss)

In [ ]:
test_near(F.nll_loss(F.log_softmax(pred, -1), y_train), loss)

In [ ]:
test_near(F.cross_entropy(pred, y_train), loss)

## Basic training loop

In [ ]:
loss_func = F.cross_entropy

In [ ]:
#export
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

In [ ]:
bs=64                  # batch size

xb = x_train[0:bs]     # a mini-batch from x
preds = model(xb)      # predictions
preds[0], preds.shape